<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_58.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4

In [ ]:
# !unzip "/content/joint_representations.zip"

In [ ]:
# !gdown https://drive.google.com/uc?id=1wgl3QGXZ4m2aLg3T-1TDXQqSP31RuXgL

In [ ]:
# !unzip "/content/hateful_train+test_unseen.zip"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Load Model


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-14/checkpoint-42000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-14/checkpoint-42000")

loading configuration file /content/Model/Models-Train-14/checkpoint-42000/config.json
Model config ViTConfig {
  "_name_or_path": "/content/Model/Models-Train-14/checkpoint-119000_f",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
 

# Standard Experiment

In [4]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-food-101-test", split="train")

Resolving data files:   0%|          | 0/22716 [00:00<?, ?it/s]

In [5]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

# 100% Missing Modality Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
for i in range(150,500):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [ ]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-food-101-test", split="train")

In [ ]:
ds_f

In [ ]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.7

In [ ]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

In [ ]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

In [ ]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [ ]:
# selected_indices

In [ ]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [ ]:
# selected_indices

In [ ]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [ ]:
fused_ds

In [ ]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [ ]:
ds_imgs = ds_joint.filter(filter_funtion)

In [ ]:
ds_imgs

In [ ]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [ ]:
all_indices = range(len(ds_imgs))
print(all_indices)

In [ ]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [ ]:
# remaining_indices

In [ ]:
missing_ds = ds_imgs.select(remaining_indices)

In [ ]:
missing_ds

In [ ]:
# Combine Both Datasets

In [ ]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [ ]:
ds

# Pre-Processing and Validation Loop

In [6]:
labels = ds.features["label"]
labels

ClassLabel(names=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'mac

In [7]:
labels.int2str(ds[532]["label"])

'baklava'

In [8]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [9]:
dataset = ds.with_transform(transform)

In [10]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [11]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [12]:
labels = ds.features["label"].names
labels

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [13]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [14]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [15]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [16]:
batch_size = 16

In [17]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [18]:
model = model.to(device)
model.eval()
predictions, labels = [], []
valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    loss = outputs.loss
    valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [20]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.9019193519985913
precision:  0.9023828070166671
f1_score:  0.9016506960687902


In [21]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[214   0   1 ...   0   0   1]
 [  0 208   0 ...   0   0   0]
 [  1   0 221 ...   0   0   0]
 ...
 [  0   0   0 ... 225   0   0]
 [  0   0   0 ...   0 168   0]
 [  1   0   0 ...   0   0 207]]
